Required packages

In [12]:
from sor import SOR
import os, shutil
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
import openpyxl

Before running, make sure the table files containing original frame-wise feature arrays are properly prepared and stored in the /1.FormantTable folder. Furthermore, you need make 5 new folders at the same root directory and name them 2.FormantExcel 3.DerivedMetric 4.SDRdata 5.PCAmodels 6.OmicsData respectively.

In [3]:
cpath='C:\\Users\\surgi\\source\\repos'
path=cpath+'\\1.FormantTable'
outpath=cpath+'\\6.OmicsData'

The following two cells can execute normally if you have already prepared the 10000 iterations for FCR, VSA and F2R, 
and the data files are sored in 3.DerivedMetric folder

In [6]:
pdn=pd.read_csv(cpath+'\\3.DerivedMetric\\data10000_0.2.txt',sep='\t',index_col=0)
case=pdn.index.to_list()
pdr=pd.read_excel(cpath+'\\0.basic\\pdr.xlsx',index_col=0)

In [7]:
for th in tqdm(range(2, 10)):
    threshold = th / 10
    pdn = pd.read_csv(cpath + '\\3.DerivedMetric\\data10000_{}.txt'.format(str(threshold)),sep='\t',index_col=0)

    for thr in tqdm(range(10, 101, 10)):
        interval = thr
        epo = SOR.SOR(path=path, outpath=outpath, case=case, thresholds=threshold, intervals=interval, pdr=pdr)
        pdl = epo.get_pdl(pdn, spath=cpath + '\\4.SDRdata')
        npd0, pddata = epo.get_ALL(pdl, spath=cpath + '\\4.SDRdata')
        epo.PCA(npd0, pddata, ppath=cpath + '\\5.PCAmodels')
    time.sleep

ofile=[]
for file in os.listdir(cpath + '\\6.OmicsData'):
    of0=pd.read_csv(cpath + '\\6.OmicsData\\'+file,sep='\t',index_col=0)
    ofile.append(of0)
pan_file=pd.concat(ofile,axis=1)
pan_file.to_csv(cpath+'\\Speech_Omics_data.txt',sep='\t',encoding='utf_8_sig')

100%|██████████| 8/8 [4:41:33<00:00, 2111.65s/it]


If you need to restart 10000 iterations of computing FCR, VSA and F2R, please run the following two cells.

In [ ]:
case = list(set([fn[4:21] for fn in os.listdir(path)]))
pdr = pd.read_excel(cpath + '\\0.basic\\pdr.xlsx', index_col=0)

In [ ]:
epo = SOR.SOR(path=path, outpath=outpath, case=case, thresholds=1, intervals=1, pdr=pdr)
pkde = epo.get_kde(kpath=cpath + '\\2.FormantExcel')
for th in tqdm(range(2, 10)):
    threshold = th / 10
    epo = SOR.SOR(path=path, outpath=outpath, case=case, thresholds=threshold, intervals=1, pdr=pdr)
    pdn = epo.get_pdn(ipath=cpath + '\\3.DerivedMetric') # This step is time-consuming.

    for thr in tqdm(range(10, 101, 10)):
        interval = thr
        epo = SOR.SOR(path=path, outpath=outpath, case=case, thresholds=threshold, intervals=interval, pdr=pdr)
        pdl = epo.get_pdl(pdn, spath=cpath + '\\4.SDRdata')
        npd0, pddata = epo.get_ALL(pdl, spath=cpath + '\\4.SDRdata')
        epo.PCA(npd0, pddata, ppath=cpath + '\\5.PCAmodels')
    time.sleep

ofile=[]
for file in os.listdir(cpath + '\\6.OmicsData'):
    of0=pd.read_csv(cpath + '\\6.OmicsData\\'+file,sep='\t',index_col=0)
    ofile.append(of0)
pan_file=pd.concat(ofile,axis=1)
pan_file.to_csv(cpath+'\\Speech_Omics_data.txt',sep='\t',encoding='utf_8_sig')